In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('/kaggle/input/yeh-concret-data/Concrete_Data_Yeh.csv')
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
plt.figure(figsize=(8,7))
sns.heatmap(data.corr(),annot=True)
plt.show()

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

#### Data has no null values

In [ ]:
# Checking the pairplot.
sns.pairplot(data)
plt.show()

In [ ]:
# Age columns seems as it is in the form of categories

In [ ]:
data['age'].nunique()
# We'll proceed by using it as s numeric column only.

In [ ]:
# checking the distribution of the data
for i in data.iloc[:,:-1].columns:
    sns.kdeplot(data[i])
    plt.show()

In [ ]:
data.columns

In [ ]:
for i in data.iloc[:,:-1].columns:
    print(i,end=' :')
    print(data[i].nunique())

In [ ]:
# Checking for outliers in the data
for i in data.iloc[:,:-1].columns:
    sns.boxplot(data[i])
    plt.show()

In [ ]:
data1 = data.copy(deep=True)

In [ ]:
data1.isnull().sum()

# Outlier Treatment
for i in data1.iloc[:,:-1].columns:
    q1 = data1[i].quantile(0.25)
    q3 = data1[i].quantile(0.75)
    iqr = q3-q1
    ub = q3+(1.5*iqr)
    lb = q1-(1.5*iqr)
    data1[i]=data1[~((data1[i] < lb) | (data1[i] > ub))]

data1.isnull().sum()

for i in data1.columns:
    data1[i].fillna(method='ffill',inplace=True)

data1.isnull().sum()

for i in data1.iloc[:,:-1].columns:
    sns.boxplot(data1[i])
    plt.show()

for i in data1.iloc[:,:-1].columns:
    sns.kdeplot(data1[i])
    plt.show()

In [ ]:
# Checking skewness of data now
data1.skew()

## Still skewness can be corrected using the Power Transformer

In [ ]:
from sklearn.preprocessing import PowerTransformer

In [ ]:
pt = PowerTransformer()
X = pt.fit_transform(data1.iloc[:,:-1])
X = pd.DataFrame(X,columns=data1.iloc[:,:-1].columns)
X.head()

In [ ]:
X.skew()

In [ ]:
Y = data[['csMPa']]

In [ ]:
# Final Data prepared for models is
# Input Variables X and Target column Y.

In [ ]:
import statsmodels.api as sm

In [ ]:
inp = X
out = Y
c = sm.add_constant(inp)
model = sm.OLS(out,c).fit()
model.summary()

In [ ]:
import scipy.stats as stats

In [ ]:
stats.probplot(model.resid,plot=plt)

In [ ]:
# Checking Normality of the residue
sns.distplot(model.resid)
plt.show()

# Data can be seen somewhat normal.

In [ ]:
model.resid.skew()   # Skewness of residue is also under control.

In [ ]:
# Checking Multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(inp.values,i) for i in range(inp.shape[1])]
vif['Features'] = inp.columns
vif.sort_values('VIF',ascending=False)

In [ ]:
# Multicollinearity is also under check.

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split, KFold, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
# Splitting the dataset
xtrain, xtest, ytrain, ytest = train_test_split(X,Y,test_size=0.25,random_state=0)

In [ ]:
LR = LinearRegression()
LR.fit(xtrain,ytrain)

In [ ]:
LR.score(xtrain,ytrain)

In [ ]:
LR.score(xtest,ytest)

In [ ]:
ytrain_pred = LR.predict(xtrain)
ytest_pred = LR.predict(xtest)

r2_train = r2_score(ytrain,ytrain_pred)
r2_test = r2_score(ytest,ytest_pred)

rmse_train = np.sqrt(mean_squared_error(ytrain,ytrain_pred))
rmse_test = np.sqrt(mean_squared_error(ytest,ytest_pred))

print('R2 Train: ',r2_train,end='    ')
print('R2 Test: ', r2_test)
print('RMSE Train: ',rmse_train,end='    ')
print('RMSE Test: ',rmse_test)

In [ ]:
kf = KFold(shuffle=True,n_splits=5,random_state=0)
score1 = cross_val_score(LR,xtrain,ytrain,cv=kf,scoring='r2')
LR_be = np.mean(1-score1)
LR_ve = np.std(score1,ddof=1)
print('Bias Error: ',LR_be)
print('Variance Error: ',LR_ve)

In [ ]:
Tab = pd.DataFrame()

In [ ]:
Tab['LR'] = [LR_be,LR_ve]

In [ ]:
Tab

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
KNR = KNeighborsRegressor()
KNR.fit(xtrain,ytrain)

In [ ]:
KNR.score(xtrain,ytrain)

In [ ]:
KNR.score(xtest,ytest)

In [ ]:
ytrain_pred = KNR.predict(xtrain)
ytest_pred = KNR.predict(xtest)

r2_train = r2_score(ytrain,ytrain_pred)
r2_test = r2_score(ytest,ytest_pred)

rmse_train = np.sqrt(mean_squared_error(ytrain,ytrain_pred))
rmse_test = np.sqrt(mean_squared_error(ytest,ytest_pred))

print('R2 Train: ',r2_train,end='    ')
print('R2 Test: ', r2_test)
print('RMSE Train: ',rmse_train,end='    ')
print('RMSE Test: ',rmse_test)

In [ ]:
# The model is highly Over fitting

In [ ]:
kf = KFold(shuffle=True,n_splits=5,random_state=0)
score2 = cross_val_score(KNR,xtrain,ytrain,cv=kf,scoring='r2')
KNR_be = np.mean(1-score2)
KNR_ve = np.std(score2,ddof=1)
print('Bias Error: ',KNR_be)
print('Variance Error: ',KNR_ve)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
RF = RandomForestRegressor()
RF.fit(xtrain,ytrain)

In [ ]:
RF.score(xtrain,ytrain)

In [ ]:
RF.score(xtest,ytest)

In [ ]:
ytrain_pred = RF.predict(xtrain)
ytest_pred = RF.predict(xtest)

r2_train = r2_score(ytrain,ytrain_pred)
r2_test = r2_score(ytest,ytest_pred)

rmse_train = np.sqrt(mean_squared_error(ytrain,ytrain_pred))
rmse_test = np.sqrt(mean_squared_error(ytest,ytest_pred))

print('R2 Train: ',r2_train,end='    ')
print('R2 Test: ', r2_test)
print('RMSE Train: ',rmse_train,end='    ')
print('RMSE Test: ',rmse_test)

In [ ]:
kf = KFold(shuffle=True,n_splits=5,random_state=0)
score3 = cross_val_score(RF,xtrain,ytrain,cv=kf,scoring='r2')
RF_be = np.mean(1-score3)
RF_ve = np.std(score3,ddof=1)
print('Bias Error: ',RF_be)
print('Variance Error: ',RF_ve)

In [ ]:
from xgboost import XGBRegressor

In [ ]:
XGB = XGBRegressor(random_state=0)
XGB.fit(xtrain,ytrain)

In [ ]:
XGB.score(xtrain,ytrain)

In [ ]:
XGB.score(xtest,ytest)

In [ ]:
ytrain_pred = XGB.predict(xtrain)
ytest_pred = XGB.predict(xtest)

r2_train = r2_score(ytrain,ytrain_pred)
r2_test = r2_score(ytest,ytest_pred)

rmse_train = np.sqrt(mean_squared_error(ytrain,ytrain_pred))
rmse_test = np.sqrt(mean_squared_error(ytest,ytest_pred))

print('R2 Train: ',r2_train,end='    ')
print('R2 Test: ', r2_test)
print('RMSE Train: ',rmse_train,end='    ')
print('RMSE Test: ',rmse_test)

In [ ]:
# Applyting Regularization

In [ ]:
from sklearn.linear_model import Ridge, Lasso

In [ ]:
# Searching for the best parameter.
rid = Ridge()
param = {'alpha':[0.0001,0.00,0.01,0.1,0.5,1,2,5,10]}
GS = GridSearchCV(rid,param,cv=5, scoring='neg_mean_squared_error')
model1 = GS.fit(xtrain,ytrain)

In [ ]:
model1.best_params_

In [ ]:
rid = Ridge(alpha=5)
rid.fit(xtrain,ytrain)

In [ ]:
ytrain_pred = rid.predict(xtrain)
ytest_pred = rid.predict(xtest)

r2_train = r2_score(ytrain,ytrain_pred)
r2_test = r2_score(ytest,ytest_pred)

rmse_train = np.sqrt(mean_squared_error(ytrain,ytrain_pred))
rmse_test = np.sqrt(mean_squared_error(ytest,ytest_pred))

print('R2 Train: ',r2_train,end='    ')
print('R2 Test: ', r2_test)
print('RMSE Train: ',rmse_train,end='    ')
print('RMSE Test: ',rmse_test)

In [ ]:
# No imporvement at all.